In [1]:
# dependencies
import pandas as pd
import requests
import json
from census import Census
from us import states

# from uszipcode import SearchEngine

# US zipcode search (simple_zipcode=False provides polygon data)
#search = SearchEngine(simple_zipcode=True)

# census API key
from config import api_key

In [51]:
years = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

for year in years:
    c = Census(api_key, year=year)

    # From API: Population, State FIPS, County FIPS
    total = pd.DataFrame(c.acs5.state_county('B01003_001E', Census.ALL, Census.ALL))
    white = pd.DataFrame(c.acs5.state_county('B02001_002E', Census.ALL, Census.ALL))

    # Rename Census variables
    total = total.rename(columns={'B01003_001E': 'total'})
    white = white.rename(columns={'B02001_002E': 'white'})

    # Create national county FIPS
    total['fips'] = total.state + total.county
    white['fips'] = white.state + white.county

    # Recast FIPS as integer
    total = total.astype({'fips': 'int32'})
    white = white.astype({'fips': 'int32'})

    # Merge into single dataframe
    full = pd.merge(total, white, on='fips')

    # Read in county names
    county_names = pd.read_csv('project_two/static/county_fips_master.csv', encoding = "ISO-8859-1")
    county_names = county_names[['fips', 'county_name', 'state_name']]
    county_names = county_names.astype({'fips': 'int32'})

    # Merge county names into data
    fullCounties = pd.merge(full, county_names, on='fips')

    # Clean data and extract necessary columns only
    fullCounties['white_pct'] = round((fullCounties.white/fullCounties.total*100), 2)
    fullCounties['nonwhite_pct'] = 100 - fullCounties.white_pct
    fullCounties['year'] = year
    fullCounties = fullCounties[['fips', 'county_name', 'state_name', 'nonwhite_pct', 'year']]
    
    # Export to JSON format
    fullCounties.to_json(f'project_two/data/census{year}.json', orient='records')